In [ ]:
import sys
sys.path.append('../src')

In [ ]:
import pandas as pd

from sklearn.metrics.scorer import f1_scorer, roc_auc_scorer
from sklearn.model_selection import RandomizedSearchCV

import parameters
import model_io
from model_definitions import model_packages

In [ ]:
# Build Training Set
# train_df = 

In [ ]:
train_df.head()

In [ ]:
def get_best_model_from_result_dict(results_dict, scorer):
    get_best_score = lambda rndm_cv_tup: scorer._sign*rndm_cv_tup[1].best_score_
    return max(results_dict.iteritems(), key = get_best_score)

In [ ]:
n_configs = 40
n_folds = 5
scorer = f1_scorer

results = dict()
for name, model_pipeline, param_grid in model_packages:
    
    rndm_cv = RandomizedSearchCV( 
                        estimator = model_pipeline, 
                        param_distributions = param_grid, 
                        n_iter = n_configs,
                        n_jobs = -1,
                        cv = n_folds,
                        random_state = 1492,
                        scoring = scorer,
                        verbose = -2,
                      )
    
    input_columns = filter(lambda s: s != parameters.OUTCOME_COLUMN_NAME, list(train_df.columns))
    rndm_cv.fit(train_df.loc[:, ], y = train_df.loc[:, parameters.OUTCOME_COLUMN_NAME])
    results[name] = rndm_cv
    
best_model_name, best_rndm_cv = get_best_model_from_result_dict(results, scorer)    

In [ ]:
model_io.save_model(best_model_name, best_rndm_cv.best_estimator_)